
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Project of basics of mobile robotics - Thymio in Paris<br/> Autumn 2021 <br/> </strong><br/>

<div style="justify;text-justify: inter-word">
Bonvalot Isione <br>
Franca Sébastien <br>
In-Albon Malika <br>
Pellegrini Sylvain <br>
</div>



---
# Introduction 


## Context
During the course of Basics of Mobile Robotics, we were asked to complete a project combining five modules: Vision, Global Navigation, Local Navigation, Filtering and Motion Control. First we created an environment into which the robot can navigate. The objective is to cross under a bridge, go the the final goal and during that time avoid obstacles. The first part consists in getting the position of the elements in the setup: it is the vision part. Then the shortest path is computed thanks to the Global Navigation element. If an obstacle pops during the displacement of the robot, the Local Navigation will take care of avoiding it. The Filtering part predicts the position of the robot when the Vision part looses the robot's position. The Motion Control part computes the error between the robot's position and the path that he should follow. It controls the speed of the robot given the information of the different parts combined.

<p align="justify"> The story behind the project is the following. Thymio is on holiday in Paris. He had a long day, so he intends to stroll down the Champs Elysées and under the Arc of Triomphe by the shortest path through traffic. Then, Thymio wants to enjoy a view of the Eiffel Tower, still by the shortest path. During his walk, some Parisians in a hurry crosses his road but thanks to his good ability, Thymio avoids them and continues along the streets. </p>

## Environment set-up
<p> The ground where Thymio navigates is white to make the different elements of the setup appear more clearly. The components are listed below:</p>

<ul>
    <li> <b>Robot:</b> two blue circles on top, smaller circle defines the front of the robot
    <li> <b>Goal:</b> 2D green circle</li>
    <li> <b>Arc de Thriomphe:</b> 3D rectangle obstacle in red with brown sides. Thymio should pass under this obstacle</li>
    <li> <b>Obstacles:</b> 2D triangles or squares are black</li>
    <li> <b>Unforeseen obstacles:</b> 3D white cylinder</li>        
</ul>

<figure>
  <img src="images/report/original.jpg" style="width:40%">
  <figcaption>Fig.1 - Top view of the setup</figcaption>
</figure> 

The decision of making the obstacles in two dimensions has been taken to avoid conflict between local avoidance and global navigation. This way, Thymio can go very close to the permanent obstacles without detecting it and at the same time detect the unforeseen obstacles without trouble.

<p> The camera used for the project is the camera provided in class. It is the AUKEY Overview Full HD Video 1080p Webcam. The camera is hung to a lamp on top of the set-up.</p>


# Vision

## Get images and obstacles
The external camera capture is initiated using the opencv library. Since the number assigned to an external camera changes with users, you should always check that the camera selected in the code is the right one. The CAP_DSHOW argument is very important as it drastically reduces the connection time to the camera (from 2 minutes down to less than a second).

The vision first starts by extracting the obstacles from the red channel (3d dimension index 2, as the picture is in BGR format). It blurs it and then performs an opening. This filter consists of an erosion followed by a dilation. This reduces the noise in the image. The final pre-processing consists of a thresholding. The filtered image is then fed to the opencv findContours() function and the result is stored as an array of contours. They are then approximated to reduce computational cost and simply the obstacle assessment. Two characteristics are judged to classify a contour as an obstacle : a perimeter length neither over nor under what is expected, and the number of vertice must be contained between 3 and a maximum determined experimentally to take shadows into account.

Next, using the extracted obstacles, it expands them by moving each vertex by a constant times the vector connecting it to the center of the obstacle. The center is found using the same equation using moments that will be presented in the next chapter. This is done to take Thymio’s width into account in the computations. Finally, the module also puts them in a slightly different format that fits the input requirements for the global pathing functions.

The functions used to annotate a rough version of the environment are also included in the vision source file. They consist only of a combination of opencv drawing utilities that act on the canvas image provided. 


## Position of robot, arch and goal
To detect the robot, arch and goal positions, a color and shape code has been defined. The robot is represented by two blue circles of different sizes, the red rectangle corresponds to the arch and the final goal is represented by a green circle. First of all, the function cv.inRange() of the opencv library produces some threshold images for each color. Then the different shapes are extracted to get the positions.


### Find robot and goal position
Detecting circles is achieved with the function cv.HoughCircles which requires a greyscale image as an input. Several conversions need to be done because the cv.inRange function needs a hsv image in input and delivers a threshold image as an output. As shown in Figure 2, the mask is then applied to the greyscale image. It is useful to blur the image before inserting it in the HoughCircles function, because it helps smoothing the contour of the masked image. The output is an array containing the circles center and radius.

<figure>
  <img src="images/report/frame-grey_mask.png" style="width:100%">
  <figcaption>Fig.2 - Mask applied to grayscale image</figcaption>
</figure>

To get the goal position, the function get_goal_position() returns the circle's center and radius. The green circle is detected with the detect_circles() function, which directly returns the center and the radius of the circle. The center is used to give the (x,y) coordinates of the goal position and the radius will be used in the function convert_meter2pxl() later in the project.

<figure>
  <img src="images/report/frame-goal.png" style="width:100%">
  <figcaption>Fig.3 - Different steps to get the goal position</figcaption>
</figure>

To return the robot position, the function get_robot_position() returns not only the (x, y) coordinates, but also an angle to give the direction of the thymio. First of all, the function detect_circles() is called to get the two circles center and radius. It has been defined that the smaller circle defines the front of the robot. The function calculates the angle between the two centers. The function finally returns the (x, y) coordinates of the larger circle and the angle between the two circles.


<figure>
  <img src="images/report/frame-robot.png" style="width:100%">
  <figcaption>Fig.4 - Different steps to detect the two circles of the robot</figcaption>
</figure>

### Find "Arc de Triomphe" position
The function get_arch_position() return two points (point1, point2) along which the robot should pass to go through the arch as shown in Figure 4. First of all, a threshold image on the red color is created with the cv.inRange() function, then the image is blurred.

<figure>
  <img src="images/report/frame-arch.png" style="width:100%">
  <figcaption>Fig.4 - Different steps to detect the arch and print point1 and point2</figcaption>
</figure>

The next step is finding the rectangle, it is done with the function cv.findContours() and cv.approxPolyDP() of the opencv library. To find point1 and point2, it is necessary to compute the center and the orientation of the arch. To do so, the image moment $M_{1,0}$, $M_{0,0}$, and $M_{0,1}$ are given with the function cv.moments of the opencv library. It is then possible to calculate the barycenter and the angle of the rectangle, the angle returned is between -90° and 90° counterclockwise.

First the following formula gives the coordinates of the barycenter.

<center> $cx = \frac{M_{1,0}}{M_{0,0}}$ and $cy = \frac{M_{0,1}}{M_{0,0}}$

The angle is finally given by this formula:

<center> $\theta = -\frac{1}{2}\ tan^{-1}\left( \frac{2\mu'_{1,1}}{\mu'_{2,0}\ \mu'_{0,2}}\right)$ with <br> <right> $\mu'_{2,0} = \frac{M_{2,0}}{M_{0,0}}-cx^2,\quad \mu'_{1,1} = \frac{M_{1,1}}{M_{0,0}}-cx\cdot cy,\quad \mu'_{0,2} = \frac{M_{0,2}}{M_{0,0}}-cy^2$
    
Knowing the center of the arch and its orientation, it possible to find point1 and point2 with trivial trigonometry calculations.

 ### Sanity check 
All the function before return a boolean success value, additionaly to the value they should return. For instance if the functions detect more than one or two circles, if the arch is not detected as a retangle, etc.. It is useful to avoid problems because the vision does not always find the shapes due to inexactitudes. So in the while(True) there is no problem if sometimes the function return False, None. 



## Printing images of Thymio, Eiffel Tower and Arc de Triomphe
To illustrate the project Thymio in Paris, the objective was to print images of Thymio, Arc de Triomphe and Eiffel Tower in the « mission status » image instead of drawing simple shapes like circles and arrows. However, a problem appeared because the time of execution of the loop took three times longer than usual (0.4 seconds instead of 0.15 seconds). It means that we had to adapt the refresh rate Ts of the Kalman filter which was too slow. The final decision was printing the fancy images only in the “environment” image during the state COMPUTE and keep the simple shapes drawing in the “mission status” image for the state TRAVEL.

To print the image of the Arc de Triomphe and Eiffel Tower, there was no difficulty because only the x, y position had to be taken in account.  However, to print the robot not only the x, y position was important but also the angle of the robot. The first step is rotating the image of the Thymio with the functions cv.getRotationMatrix2D() and cv.warpAffine() of the opencv library. The second step is to adjust the position of the robot in the provided image to perfectly match the real robot position.

<figure>
  <img src="images/report/environment.jpg" style="width:40%">
  <figcaption>Fig.5 - Environment image with images of Thymio, Eiffel Tower and Arc of Triomphe printed</figcaption>
</figure>


---
# Navigation

## Global navigation

**For simplicity, the lists' indices used in code for the Global Navigation module follow this convention: <br>
index 0 corresponds to start, maximal index corresponds to goal, and all intermediary indices correspond to obstacle vertices. <br>
This allows us to work with points' indices (also referred to as names) rather than points' coordinates.**
<br/>

### Visibility graph
After comparing the available options for creating a connected graph, we chose to use a visibility graph. In fact, a grid search does not appear as a good choice here because of the image distortion (due to the position of the camera), the difficulty of controlling the Thymio in a grid rather than in coordinates, the need for arbitrary parameters (e.g. size of the grid), and the heavyness of computation associated to the entire grid. In contrast, a visibility graph is quickly built once we know the obstacles' coordinates, and its implementation with the motion is straightforward due to the use of coordinates rather than a grid.



The code for computing the visibility graph given all coordinates is given in global_path.py.<br/>
The form of the output is a list of all points' neighbours' name and their distance to that point: <br>
[[neighbours_of_start], [neighbours_of_vertex1],..., [neighbours_of_goal]] <br>
It is used as the input for the shortest path algorithm.

#### Completing the list of neighbours

<br/>

We want to indicate for each point:
- who its neighbours are
- how far they are from that point<br>

*(Note that the term "neighbour" refers to all points that are visible by the current point, i.e. no line connecting two neighbours passes trough an obstacle.)*
<br/><br/>

To simplify our search, we assume that our obstacles are simple shapes such as squares or triangles. Thanks to this assumption, we only have five different possibilities to determine whether two points p1,p2 are neighbours (their connection line **CL** is valid) or not: <br>

1. (p1,p2) is a side of an obstacle $\rightarrow$ CL stays valid (segment v1 on Figure 6)<br>
2. p1 and p2 lie on the same obstacle, but CL is not a side of the obstacle $\rightarrow$ CL becomes not valid (segment nv1 on the Figure 6)
3. p1 and p2 do not lie on the same obstacle. p1 and/or p2 lie on all lines that CL intersects $\rightarrow$ CL stays valid (segment v2 on Figure 6)
4. p1 and p2 do not lie on the same obstacle. CL intersects some lines on which neither p1 nor p2 lie $\rightarrow$ CL becomes not valid (segment nv2 on Figure 6)
5. Other cases: we consider that CL stays valid (this did not cause any problems upon testing).

Note that this logic is not robust to complicated obstacle shapes. On the following figure, nv3 would not be valid due to condition 2. However, this logic is sufficient with the kind of obstacles that we use.<br>
<br/>


#### Overlaps
<br/>
The obstacles used for the algorithm are enlarged versions of the real obstacles, to allow Thymio to pass in their neighbourhood without hitting them. This means that there is a risk of overlap between two obstacles.<br>
To take it into account, we create a list with all obstacle vertices located within another obstacle (e.g. point 9 on Figure 6). If a point is within this list, we will bypass the above conditions and force it to have no neighbours.




<figure>
  <img src="./images/VisibilityGraph.JPG" alt="Trulli" style="width:50%">
  <figcaption>Fig.6 - Result of the visibility graph code</figcaption>
</figure
    
<br/>
Note the indexing convention: point 0 is the start, point 15 is the goal, all intermediary indices are obstacles' vertices. <br/>

Our algorithm considers that v1 and v2 are valid (conditions 1 and 3 respectively). <br>
nv1,nv2,nv3 are not valid (conditions 2, 4, 2 respectively).

The situation for nv4 is a bit more complex. It is a side of obstacle 12-13-14, hence considered valid at first (condition 1). But once we check nv4 with obstacle 9-10-11, it is considered invalid (condition 4). In conclusion, nv4 is considered not valid.

For one point and one obstacle, the pseudo-code that implements this logic is as follows:

### Optimal path finding

Now that the visibility graph is known, we can explore it to determine which path leads to the goal in an optimal way. This relies on the A* algorithm (the logic and pseudo-codes used in this section are inspired from the lecture slides and the Wikipedia page on A* algorithm: https://en.wikipedia.org/wiki/A*_search_algorithm, *cons. 08.12.2021*). <br/>

The developed code can apply whether the Dijkstra (in case that the start and goal position are not specified) or the A* algorithm (used heuristic: Euclidean distance from the goal). For the final implementation we went with the A* algorithm, but both methods yielded similar results with no major difference in execution time. <br/>

The code for finding the optimal path is given in shortest_path.py. <br/><br>

#### Initialization

The initialization follows this order:<br>
1. Calculate the visibility graph thanks to global_path.py
2. Create a list with all points' names: [0, 1, ..., N, N+1] (following the convention 0$\rightarrow$start and N+1$\rightarrow$goal)
3. Assign the heuristic to every point of the graph (0 if Dijkstra algorithm is used ; Euclidean distance to the goal if A* algorithm is used).
4. Attribute infinite weight to every point of the graph, except the start (weight_start = 0 + heuristic)
5. Initialize the following lists: 
    - predecessors, where we will store pieces of the optimal path before reconstructing it entirely. For example: "To reach goal, come from vertex 5. To reach vertex 5, come from vertex 2. To reach vertex 2...".
    - vertices_to_explore, which will indicate what points on the graph are still unexplored. <br/>


#### Graph exploration <br/>

The pseudo-code for graph exploration is given in the following cell. A few precisions:
- current_neighbours contains the names of current_point's neighbours, but also their distance from current_point
- predecessors is the list where pieces of the optimal path are stored for later reconstruction

#### Reconstructing the path

Once the list *predecessors* is filled, we work backwards to reconstruct the entire path until the start is reached. <br>
For example, let's say predecessors = ["reached_start", 0, 0, 2, 1, 3]. The goal (index 5) is reached from index 3. Index 3 is reached from index 2. Index 2 is reached from index 0 (start). <br>
The optimal path is then: [0, 2, 3, 5].

## Local navigation </br>

<p> <i> The code was inspired by the exercises sessions 3 and 4 from the course MICRO-452 : Basics of mobile robotics. </i> </p>

<p> The local navigation is used to avoid physical obstacles that are unpredictible. These obstacles can be put in Thymio's path at any point in time so that the Thymio should leave the optimal path to avoid them. Once the obstacle is passed, the robot has to go back to the initial optimal path. This is done by the motion control module. There are many possibilities to implement a local avoidance using the proximity sensors. </p> </br>

### Simplified ANN </br>

#### Obstacle detection
<p>To detect an obstacle, the five front proximity sensors are used. 
<br/>
    
<figure>
  <img src="images/robot_sensors.png" style="width:30%">
  <figcaption>Fig.7 - Caption - source : MICRO-452 moodle</figcaption>
</figure>

<br/></p>
<ul>
    <li> To detect an obstacle on the left side : we calculate the mean of the first and second sensors.</li>
    <li> To detect an obstacle on the right side : we calculate the mean of the fourth and fifth sensors. </li>
    <li> To detect an obstacle in front of the robot : we measure the value of the third sensors.</li>
</ul>
</br>

<p> The code to check if an obstacle is detected is given below. </p>

In [ ]:
OBST_THR_L = 10      # low obstacle threshold 
OBST_THR_H = 60      # high obstacle threshold 

def check_obstacle(prox_sensors):
    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_sensors[0], prox_sensors[1], \
            prox_sensors[2], prox_sensors[3], \
            prox_sensors[4]]
    mean_obst_left = (obst[0] + obst[1])//2
    mean_obst_right = (obst[3] + obst[4])//2
    obst_front = obst[2]
    
    if (mean_obst_right > OBST_THR_H) or (mean_obst_left > OBST_THR_H) \
        or (obst_front > OBST_THR_H) :
        obstacle = True
    else:
        obstacle = False
    return obstacle, obst_front, mean_obst_left, mean_obst_right


#### Pseudo-code
 <p> The following pseudo-code describes the different steps once an obstacle is detected somewhere in front of the robot.  </p>
<p> There are two cases depending on the position of the unforseen obstacle. 
<ul>
<li>Case 1 : Obstacle only in front of the robot (detected only by the third sensor) </li>
    <ul>
    <li> The robot turns in place to the left (tester de randomiser la direction qu'il prend ?</li>
    <li> The speed is constant </li>
    </ul>
<li>Case 2 : Obstacle detected on either the left or the right side</li>
    <ul>
    <li> The robot accelerates in the opposite direction of the obstacle</li>
    <li> The speed depends on the values of the sensors means </li>
    </ul>
</ul>  

For the first case, to ensure that the robot moves straight forward to the obstacle, we check that the values of the wheels are almost the same. (tournure de phrase pas ouf je trouve)
</p>

The code concerning the case 1 is given below.

In [ ]:
if (abs(motor_left_speed - motor_right_speed) < 50):
    if obst_front > OBST_THR_H:
        motor_left = -SPEED0
        motor_right = SPEED0
    else:
        obst_in_front = 0

The code concerning the case 2 is given below.

In [ ]:
if obst_in_front == 0:
    motor_left = SPEED0 + OBST_SPEED_GAIN*(mean_obst_left//100)
    motor_right = SPEED0 + OBST_SPEED_GAIN*(mean_obst_right//100)

---
# Motion Control

## Proportional controller 
<p> The robot's motion is controlled via a simple proportional controller. The idea is to calculate the angle error between the robot's position and its next goal's position. The next goals correspond to the different vertices given by the global path navigation. The error calculation is done by the function "get_error(robot_pos, next_goal)" in the motion_control.py file. </p>
<p> Given the error, the wheels' speed need to be updated according to it. Compared to a classic proportional controller, ours is a little bit upgraded. In fact, we decided to divide the NORMAL_SPEED by the error in order to reach the good path in a smoother way; If the error is large, the robot will almost turn in place to reach the good path and if the error is small, the robot will correct its trajectory while going forward. This will allow the robot to have a smooth movement.
This is done by the code given below in the function speed_control(err) of the motion_control.py file. </p>

In [ ]:
    motor_left = NORMAL_SPEED/max(1,10*abs(err)) - KP*err
    motor_right = NORMAL_SPEED/max(1,10*abs(err)) + KP*err

---
# Kalman filtering
## General structure

A Kalman filtering is introduced in the project to enhance the position & angle tracking of the Thymio. We chose this kind of filtering because it relies on the assumption that its parameters are Gaussian random variables, which does not seem far from reality. Indeed, the vision will return the position of Thymio with some uncertainty but the majority of samples will fall around the same value. The same goes for values read from odometry. 
<br><br/>
The logic implemented is very simple: knowing the state (position & angle) at step k-1 and the values returned from Thymio's odometry, predict the state at step k. If the vision is available, then also proceed to the update. The corresponding pseudo-code is given below.

The general dynamic model used is the following:

$\mathbf{x}_{k+1} = A\, \mathbf{x}_{k} + T_s \cdot \mathbf{f}(\theta_{k+1})\cdot \,[v_{\text{right}}, v_{\text{left}}]^T \\
\mathbf{y}_{k+1} = C \, x_{k}$

<br/> with:<br>
- $\mathbf{x}_k$ the state $[x,y,\theta]^T$ at step $k$;
- $\mathbf{f}$ a non-linear function which comes from the differential drive model;
- $T_s$ the sampling time in seconds;
- $v_{\text{right}}, v_{\text{left}}$ the linear speed of wheels in pixels/s ; 
- $A = C = I_{3\times3}$ equal to the identity matrix (we assume no dependance between the states). <br/> <br/>

To get $v_{\text{right}}$ and $v_{\text{left}}$, the value read by odometry must be converted from its initial unit (which seems arbitrary) into m/s, then into pixels/s. The first conversion is non-linear, hence we rely on interpolation (with scipy.interpolate.interp1d) on measures taken in a 2.5m long corridor. The second conversion is assumed linear, so that we only need to multiply by a conversion factor (given by the vision upon initialization).


## Prediction step

### From the non-linear field $f(\theta_k)$ to an input matrix $B$

As described above, we use the differential drive model:<br>
$ \begin{bmatrix} x \\ y \\ \theta \end{bmatrix}_{k+1} 
= \frac{1}{2} \begin{bmatrix} cos(\theta_{k+0.5}) & cos(\theta_{k+0.5})\\
-sin(\theta_{k+0.5}) & -sin(\theta_{k+0.5})\\
\frac{1}{L} & -\frac{1}{L}\end{bmatrix}  
\cdot \begin{bmatrix} v_{\text{right}} \\ v_{\text{left}}\end{bmatrix}$ <br>
with L the axle length of Thymio, and $\theta_{k+0.5}$ a notation used for $\frac{\theta_k + \theta_{k+1}}{2}$ (which should be more precise than using $\theta_k$ or $\theta_{k+1}$ only). <br><br/>

The function $\mathbf{f}$ represented by the 3-by-2 matrix is nonlinear and depends on the current state. However since $\theta_k$ is known, it is possible to get an estimation of $\mathbf{f}$ at each calling of the Kalman filter. To do so, we assume the following relation between $\theta_{k}$ and $\theta_{k+1}$: <br>
$\theta_{k+1} = \theta_k + T_s \cdot \frac{v_{\text{right}}-v_{\text{left}}}{2L}$

Leading to this new linearized but time-dependent model:
$\mathbf{x}_{k+1} = A\, \mathbf{x}_{k} + T_s \cdot B_{k+1} \cdot \,[v_{\text{right}}, v_{\text{left}}]^T \\
\mathbf{y}_{k+1} = C \, x_{k}$

where $B_{k+1}$ is the value of $\mathbf{f}$ calculated from $\theta_{k+0.5}$.

### Take into account the unperfect rolling of the wheels

To predict Thymio's next position, we need to estimate the error due to unperfect rolling of the wheels. We assume that $v_{\text{right}}, v_{\text{left}}$ are independent and each follow this model: <br>
$v = v_0 + \delta \; \; \;$  with $\delta \sim \mathcal{N}(0, \sigma_v)$ <br/>

To transcribe this model into Python, we initialize a matrix Q where $\delta_{\text{right}}$ and $\delta_{\text{left}}$ are stored: <br>

    Q = np.random.normal(0.0, STD_WHEELSPEED, (2,1))

### Predict the next state, calculate the covariance

Now that the matrix $B$ and the process noise are defined, we can give the new value of the state and the covariance matrix based on the lecture slides: <br>
$x_{k+1} = A\cdot x_k + T_s\cdot B_{k+1}\cdot [v_{\text{right}}, v_{\text{left}}]^T \\
\Sigma_{k+1} = (A^T \cdot \Sigma_k \cdot A^T) + \text{diag} (B\cdot Q)$ <br>
where diag$(B\cdot Q)$ is used to go from the space of wheel uncertainty to the space of covariance.

## Update step

### Measure noise

Once again, we assume that $x,y,\theta$ are independant. The matrix that represents measure noise hence is diagonal. Its components follow the distributions $\mathcal{N}(0,\sigma_i^2)$ where $\sigma_i$, $i = 1,2$ is the standard deviation introduced upon measuring coordinates ($i=1$) or $\theta$ ($i=2$).<br>

    R = np.diag([np.random.normal(0.0, STD_MEASURE_COORDS), \
                 np.random.normal(0.0, STD_MEASURE_COORDS), \
                 np.random.normal(0.0, STD_MEASURE_THETA)]) #measure noise

Note that we trust the measures from camera more than the prediction made with our model. Hence, the standard deviations used for R should be lower or equal to the one used for Q.

### Calculation

This is done according to the lecture slides:<br>
$i = y_{k+1} - Cx_{k+1}^{\text{predicted}} \\
S = (C \cdot \Sigma_{k+1}^{\text{predicted}} \cdot C^T) + R \\
K = \Sigma_{k+1}^{\text{predicted}} \cdot C^T \cdot S^{-1} $ <br/>
$x_{k+1}^{\text{updated}} = x_{k+1}^{\text{predicted}} + K\cdot i\\
\Sigma_{k+1}^{\text{updated}} = (I_{3\times 3} - KC) \Sigma_{k+1}^{\text{predicted}}$<br>
where $i$ is the innovation, $y$ is the state measured by the camera, and K is the optimal gain.

## Improving the robustness

Upon testing, the implementation of the Kalman filter seemed fine overall, but it sometimes happened (for one frame) that the Thymio thought that its position was on the other side of the map, or that it just flipped 180°. This is why we introduced some conditions on the update step, depending on the robot's history.

### Calibration

To calibrate the Kalman filter, we made Thymio run into circles. Then we added a correction factor on angle and position on the model, and we calibrated these correction factors until the trajectory predicted by Thymio was close to its real trajectory. <br>
After this calibration, our prediction model gave very satisfactory results.

### Update only if the measure makes sense

Sometimes, the vision encounters a problem for just one frame, estimating a wrong position and/or angle for the robot. Hence we implemented a simple logic: if the Thymio does not seem to have teleported then proceed to prediction and update. Otherwise, only proceed to prediction. <br><br/>
Implementing this condition forces us to store the history of positions and angles. The average of the history *avg_history* is calculated, then the update is considered only if the Euclidean distance *d(avg_history.coords, update.coords)* and the absolute difference *abs(avg_history.theta - update.theta)* are within some thresholds. <br><br/>

Implementing this logic considerably improved the smooth tracking of the position, by ignoring isolated wrong measures from the camera.

### If the Thymio is lost, then trust the vision

The problem of using these thresholds is that if the Kalman model gets too far from reality, then the values will always fall over the thresholds. To solve this issue, we implemented a second test: if the vision consistently returns the same position, we can take it as ground-truth in spite of the discrepancy with prediction. This is equivalent to bypassing the Kalman filter to force a "hard-update" with new values. <br/>

This condition also requires to store the history of error between prediction and vision's measure, as well as the history of vision's measure.

---
# Finite state machine
The main program is a loop that will go through the different states. First, it is initialised in the state COMPUTE where it assesses the environment captured by the camera and computes the optimal trajectory that the robot will have to follow. Then, when the path is calculated, the program enters the TRAVEL mode. This state will control the movement of the Thymio and, depending on the proximity sensor value, it can switch over to a local obstacle avoidance routine. When the robot reaches the last goal on his path, it enters the FINISH state and stops on the destination

# Conclusion
<p>To conclude, this project was a good way to learn how to maneuver group work.
It includes five components : vision, motion control, global navigation, local navigation, and filtering. The interconnection of the modules was the main challenge of this project. It was interesting (and a little bit stressful sometimes) to see that the modules worked fine when they were independently tested but that they failed when put together. </p>
<p> Another big hurdle to overcome during this project was the connection and the communication between the computer and the Thymio. Getting the values of the sensors and sending the instructions’ values was confusing considering the different ways to access the robot (ClientAsync, tmdclient, %%run_python, etc).</p>
<p> This project allowed us to work with an external camera which was a great learning experience because we had to tackle limitations such as image deformations, shadows or stability of the camera.
Finally, this project was fulfilling due to the multiple resources existing in python (librairies, methods,...) allowing us to build a complex program in a short span of time. </p>
<p> Coordinating work on the python files was very easy thanks to the github that was set up (which link is provided in the resources). On the other hand, working together on a jupyter notebook proved to be quite an issue as git versioning does have a hard time with the html format. This situation led us to use team management skills to communicate and work effectively as a collective unit. </p>

<p> All in all, the results obtained in this project are satisfactory but there are areas where it could still be improved upon. Indeed, some features were considered but not implemented due to the short deadline imposed. One stands out among them, which is handling the situations where a local avoidance leads the Thymio into a fixed obstacle. Even then, the implementation of theoretical models into a real-life project was definitely rewarding. </p>


github link: https://github.com/Ago-aka-pasta-boy/Thymio-in-Paris

---
# Main code to execute


In [ ]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
await node.wait_for_variables()

In [ ]:
import vision as vis
import kalman
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math
import motion_control as motion
import drawing as draw

COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

SPEED0 = 50       # nominal speed
OBST_SPEED_GAIN = 10  # /100 (actual gain: 5/100=0.05)
SENSOR_SCALE = 200  # Scale factors for sensors
MAX_SLEEP_TIME = 1
SLEEP_STEP = 0.2

OBST_THR_L = 10      # low obstacle threshold 
OBST_THR_H = 60      # high obstacle threshold 

state = 0          # 0= go to goal, 1=obstacle avoidance
obst = [0,0,0,0,0]   # measurements from the 2 left and the 2 right prox sensors

#Values for kalman filter
HISTORY_SIZE = 10 #to ignore discrete mismatch predict/update
ERR_HISTORY_SIZE = 5 #to hard-reset in case of continuous mismatch predict/update
CAM_HISTORY_SIZE = 5 #number of samples used to take the mean, in case of hard-reset

statego2goal = 0
stateobstavoid = 1

In [ ]:
# Main -> state machine
state = COMPUTE
obstacle = False
has_entered_local = 0

#valeurs pour initialiser
robot_pos_kalman = np.array([[0],[0],[0]]) #3-by-1 np.array
Sigma_kalman = np.diag([0.01,0.01,0.01])
history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE

# Read images of robot, eiffel tower and arch of triomphe
img_thymio = cv.imread('images/code/thymio.png', cv.IMREAD_UNCHANGED)
img_eiffel = cv.imread('images/code/eiffel_tower.png', cv.IMREAD_UNCHANGED)
img_arch = cv.imread('images/code/arch.png', cv.IMREAD_UNCHANGED)

cap = cv.VideoCapture(2, cv.CAP_DSHOW)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while True:
    if state == COMPUTE:
        captured, img = cap.read()
        if captured:
            # cv.imshow("Original", img)        # uncomment to debug
            cv.imshow("Original", img)
            im3 = np.zeros(img.shape, np.uint8)
            found, obstacles = vis.extract_obstacles(img)
            if found:
                ex_obstacles = vis.expand_obstacles(obstacles)
                cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                cv.imshow("expanded", im3)

            found_rob, robot_pos = pos.get_robot_position(img)
            found_goal, goal_pos, radius_pxl = pos.get_goal_position(img)
            found_arch, arch_pos = pos.get_arch_positions(img)

            if found_arch and found_rob:
                if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                    arch_pos = np.flip(arch_pos,0)

            if (found and found_rob and found_goal and found_arch):
                scale_factor = pos.convert_meter2pxl(radius_pxl)
                vis.annotate_arch(arch_pos, im3)

                AXLE_LENGTH = 0.095*scale_factor #used for kalman filter

                vertices = vis.convert_vertice(ex_obstacles)
                pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                path_arch =\
                short.pathname_to_coords(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))
                vis.annotate_path(path_arch,im3)

                pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                path_goal =\
                short.pathname_to_coords(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))
                vis.annotate_path(path_goal,im3)
                # print images of robot, eiffel tower and arc de triomphe
                draw.annotate_robot(robot_pos, im3, img_thymio, scale_factor)
                draw.annotate_eiffel_tower(goal_pos, im3, img_eiffel, scale_factor)
                draw.annotate_arch(arch_pos, im3, img_arch, scale_factor)
                cv.imshow("environment", im3)

                path = []
                path.extend(path_arch)
                path.extend(path_goal)

                #initialize kalman filter
                robot_pos_kalman = np.array([[robot_pos[0][0]],[robot_pos[0][1]],[robot_pos[1]]]) #3-by-1 np.array
                Sigma_kalman = np.diag([0.01,0.01,0.01])
                history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE
                camera_history = [np.reshape(robot_pos_kalman,(1,3))]*CAM_HISTORY_SIZE
                errpos_history = [0]*ERR_HISTORY_SIZE
                errtheta_history = [0]*ERR_HISTORY_SIZE
                Ts = 0.15

                state = TRAVEL
        else:
            print("There was a problem in the capture")
            break
    if state == TRAVEL:
        cv.imshow("environment", im3)
        cv.waitKey(0)
        
        for subgoal in path:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                prox_sensors = np.array([x for x in node['prox.horizontal'][0:5]])
                print(prox_sensors)
                obstacle, obst_front, mean_obst_left, mean_obst_right = motion.check_obstacle(prox_sensors)
                
                if obstacle == True:
                    has_entered_local = MAX_SLEEP_TIME
                    start_time = time.time()
                    # obstacle avoidance: accelerate wheel near obstacle
                    captured, img_local = cap.read()
                    im4 = np.zeros(img_local.shape,np.uint8)
                    vis.annotate_robot(robot_pos, im4)
                    cv.drawContours(im4, obstacles, -1, (0, 255, 0), 3)    
                    vis.annotate_path(path,im4)
                    cv.putText(im4, "Local avoidance", (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv.imshow("mission status", im4)
                    cv.imshow("Feed",img_travel)
                    cv.waitKey(10)
                    obst_in_front = 1
                    
                    await node.wait_for_variables({"motor.left.speed"})
                    await node.wait_for_variables({"motor.right.speed"})
                    motor_left_speed = node.v.motor.left.speed
                    motor_right_speed = node.v.motor.right.speed
                    if (abs(motor_left_speed - motor_right_speed) < 50):
                        if obst_front > OBST_THR_H:
                            motor_left = -SPEED0
                            motor_right = SPEED0
                        else:
                            obst_in_front = 0

                    if obst_in_front == 0:
                        motor_left = SPEED0 + OBST_SPEED_GAIN*(mean_obst_left//100)
                        motor_right = SPEED0 + OBST_SPEED_GAIN*(mean_obst_right//100)
                    
                    #Kalman
                    #read values from camera & sensors
                    success_camera, robot_pos_camera = pos.get_robot_position(img_local)
                    #print("robot angle", robot_pos_camera[1])
                    if success_camera:
                        camera_input = [(robot_pos_camera[0][0],robot_pos_camera[0][1]), robot_pos_camera[1], success_camera]
                        print("seen by camera", camera_input)
                    else:
                        camera_input = [(0,0),0,False]
                    await node.wait_for_variables({"motor.left.speed"})
                    await node.wait_for_variables({"motor.right.speed"})
                    motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        


                    #get position from kalman filter
                    robot_pos_kalman, Sigma_kalman, history_kalman, errpos_history, errtheta_history, camera_history =\
                    kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,\
                                        history_kalman, camera_input, Ts, scale_factor, AXLE_LENGTH,\
                                       errpos_history, errtheta_history, camera_history)           
                    robot_pos = [(int(robot_pos_kalman[0][0]), int(robot_pos_kalman[1][0])), robot_pos_kalman[2][0]]

                    Ts = time.time() - start_time
                else: #if obstacle == False                    
                    start_time = time.time()      
                    
                    captured, img_travel = cap.read()
                    cv.imshow("Feed",img_travel)
                    im4 = np.zeros(img_travel.shape)
                    cv.drawContours(im4, obstacles, -1, (0, 255, 0), 3) 
                    vis.annotate_path(path,im4)
                    vis.annotate_robot(robot_pos,im4)

                    #read values from camera & sensors
                    success_camera, robot_pos_camera = pos.get_robot_position(img_travel)
                    #print("robot angle", robot_pos_camera[1])
                    if success_camera:
                        camera_input = [(robot_pos_camera[0][0],robot_pos_camera[0][1]), robot_pos_camera[1], success_camera]
                        print("seen by camera", camera_input)
                    else:
                        camera_input = [(0,0),0,False]
                    await node.wait_for_variables({"motor.left.speed"})
                    await node.wait_for_variables({"motor.right.speed"})
                    motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        


                    #get position from kalman filter
                    robot_pos_kalman, Sigma_kalman, history_kalman, errpos_history, errtheta_history, camera_history =\
                    kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,\
                                        history_kalman, camera_input, Ts, scale_factor, AXLE_LENGTH,\
                                       errpos_history, errtheta_history, camera_history)           
                    robot_pos = [(int(robot_pos_kalman[0][0]), int(robot_pos_kalman[1][0])), robot_pos_kalman[2][0]]
                    #et apres, on passe robot_pos au controleur


                    err = motion.get_error(robot_pos,subgoal)
                    cv.putText(im4, "Global", (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv.imshow("mission status", im4)
                    cv.waitKey(10)
                    if has_entered_local <= 0.01:
                        motor_left, motor_right = motion.speed_control(err)
                        motor_left = round(motor_left)
                        motor_right = round(motor_right)
                    else:
                        motor_left = 200
                        motor_right = 200
                        #set it instantly
                        v = {
                            "motor.left.target": [motor_left],
                            "motor.right.target": [motor_right],
                        }
                        aw(node.set_variables(v))                        
                        time.sleep(SLEEP_STEP)
                        has_entered_local -= SLEEP_STEP 
                        
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
                
                Ts = time.time() - start_time
        state = FINISH
    if state == FINISH:
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        aw(node.set_variables(v))
            #print("motor_left",motor_left)
            #print("motor_right", motor_right)
            #motor_left_target = int(motor_left)
            #motor_right_target = int(motor_right)
            #print("motor_left_t",motor_left_target)
            #print("motor_right_t", motor_right_target)          
    cv.imshow("environment",im3)
    cv.waitKey(10)
cap.release()
print("finished!")
# cv.destroyAllWindows()
  
    

In [ ]:
await node.unlock()